In [2]:
import pandas as pd
import sqlite3

# Step 1: Load the CSV into a pandas DataFrame
df = pd.read_csv('/Users/josephpellumbi/Downloads/cyclistic_sql.csv', sep=';')

# Step 2: Create an in-memory SQLite database and a connection
conn = sqlite3.connect(':memory:')

# Step 3: Load the DataFrame into SQLite as a SQL table
df.to_sql('cyclistic', conn, index=False, if_exists='replace')


4143123

In [4]:
# Step 4: Count the total number of rows before deleting duplicates
initial_count_query = "SELECT COUNT(*) FROM cyclistic"
initial_count = pd.read_sql_query(initial_count_query, conn).iloc[0, 0]
print(f"Total number of rows before deduplication: {initial_count}")


Total number of rows before deduplication: 4143123


In [6]:
# Step 5: Write the SQL query to delete duplicates, keeping only the first occurrence
delete_query = """
DELETE FROM cyclistic
WHERE rowid NOT IN (
    SELECT MIN(rowid)
    FROM cyclistic
    GROUP BY ride_id
);
"""

# Step 6: Execute the delete query
conn.execute(delete_query)
conn.commit()


In [8]:
# Step 7: Count the total number of rows after deleting duplicates
final_count_query = "SELECT COUNT(*) FROM cyclistic"
final_count = pd.read_sql_query(final_count_query, conn).iloc[0, 0]
print(f"Total number of rows after deduplication: {final_count}")

# Step 8: Calculate the number of deleted duplicate rows
duplicates_deleted = initial_count - final_count
print(f"Number of duplicate rows deleted: {duplicates_deleted}")


Total number of rows after deduplication: 4143002
Number of duplicate rows deleted: 121


In [10]:
# Step 9: Retrieve the deduplicated data from the SQLite database
deduplicated_df = pd.read_sql_query("SELECT * FROM cyclistic", conn)

# Step 10: Save the deduplicated data to a new CSV file
output_file_path = '/Users/josephpellumbi/Downloads/deduplicated_file.csv'
deduplicated_df.to_csv(output_file_path, index=False)

# Step 11: Output the path to the new deduplicated file
print(f"Deduplicated data has been saved to {output_file_path}")


Deduplicated data has been saved to /Users/josephpellumbi/Downloads/deduplicated_file.csv


In [ ]:

# Step 12: Optional - Close the SQLite connection
#conn.close()